In [4]:
from datetime import datetime, timedelta
from typing import Dict, List, Optional

def get_user_meetings_in_range(user: str, start_time: datetime, end_time: datetime) -> List[Dict]:
    """Get all meetings for a user in the specified time range."""
    start_iso = start_time.isoformat()
    end_iso = end_time.isoformat()
    return retrive_calendar_events(user, start_iso, end_iso)

def find_conflicting_meetings(users: List[str], start_time: datetime, end_time: datetime) -> List[Dict]:
    """Find meetings that conflict with the requested time slot."""
    conflicting = []
    for user in users:
        user_meetings = get_user_meetings_in_range(user, start_time, end_time)
        for meeting in user_meetings:
            meeting_start = datetime.fromisoformat(meeting['StartTime'])
            meeting_end = datetime.fromisoformat(meeting['EndTime'])
            
            # Check if meeting overlaps with requested time
            if not (meeting_end <= start_time or meeting_start >= end_time):
                meeting['affected_user'] = user
                conflicting.append(meeting)
    return conflicting

def can_reschedule_meeting(meeting: Dict) -> bool:
    """Determine if a meeting can be rescheduled."""
    # Check priority if available
    if 'priority' in meeting:
        return meeting['priority'] == 0  # 0 = low priority
    
    # Check for important keywords
    subject = meeting.get('Subject', '').lower()
    important_keywords = ['ceo', 'board', 'urgent', 'critical', 'emergency', 'all-hands']
    if any(keyword in subject for keyword in important_keywords):
        return False
    
    # Check if recurring
    if meeting.get('IsRecurring', False):
        return False
    
    # Check if too close to current time (less than 2 hours)
    meeting_start = datetime.fromisoformat(meeting['StartTime'])
    if meeting_start - datetime.now() < timedelta(hours=2):
        return False
    
    return True
    
def reschedule_for_high_priority_meeting(users: List[str], start_time: datetime, 
                                       end_time: datetime, duration_minutes: int) -> Dict:
    """Reschedule existing meetings to make room for a high-priority meeting."""
    result = {'slot': None, 'rescheduled_meetings': []}
    
    # Find conflicting meetings
    conflicting_meetings = find_conflicting_meetings(users, start_time, end_time)
    
    if not conflicting_meetings:
        # No conflicts - slot is available
        result['slot'] = {
            'start_time': start_time.isoformat(),
            'end_time': end_time.isoformat(),
            'duration_minutes': duration_minutes
        }
        return result
    
    # Separate reschedulable vs non-reschedulable meetings
    reschedulable = [m for m in conflicting_meetings if can_reschedule_meeting(m)]
    non_reschedulable = [m for m in conflicting_meetings if not can_reschedule_meeting(m)]
    
    # Can't proceed if there are non-reschedulable conflicts
    if non_reschedulable:
        print(f"Cannot reschedule - {len(non_reschedulable)} high-priority meetings conflict")
        return result
    
    # Try to reschedule all conflicts
    successfully_rescheduled = []
    for meeting in reschedulable:
        alternative = find_alternative_slot_for_meeting(meeting, start_time, end_time)
        
        if alternative:
            # TODO: Call your calendar API to actually update the meeting
            # update_calendar_meeting(meeting['Id'], alternative['start_time'], alternative['end_time'])
            
            successfully_rescheduled.append({
                'meeting_id': meeting.get('Id'),
                'subject': meeting.get('Subject'),
                'original_start': meeting.get('StartTime'),
                'new_start': alternative['start_time'].isoformat(),
                'new_end': alternative['end_time'].isoformat()
            })
        else:
            # Failed to find alternative - can't proceed
            return result
    
    # Success - all meetings rescheduled
    result['slot'] = {
        'start_time': start_time.isoformat(),
        'end_time': end_time.isoformat(),
        'duration_minutes': duration_minutes
    }
    result['rescheduled_meetings'] = successfully_rescheduled
    return result

def schedule_high_priority_meeting(users: list[str], start_time: datetime, end_time: datetime, 
                                 duration_minutes: int, priority: int, free_slots: list):
    """
    Schedule meeting - will reschedule others if needed for high priority
    
    Args:
        users: List of users
        start_time: Meeting start time
        end_time: Meeting end time  
        duration_minutes: Duration in minutes
        priority: 0=low, 1=high
        free_slots: Result from find_free_slots (empty list if no slots)
        
    Returns:
        list: Free slots or rescheduled slot (never empty for high priority)
    """
    
    # If we already have slots, return them
    if free_slots:
        return free_slots
    
    # For high priority, we must find a slot by rescheduling
    if priority == 1:
        print(" High priority meeting - rescheduling others to make space")
        
        result = reschedule_for_high_priority_meeting(
            users=users,
            start_time=start_time,
            end_time=end_time,
            duration_minutes=duration_minutes
        )
        
        if result['slot']:
            print(f" Rescheduled {len(result['rescheduled_meetings'])} meetings")
            return [result['slot']]
        else:
            print(" Could not reschedule - no low priority meetings to move")
            return []  # Still return empty if genuinely impossible
    
    # Low priority meetings return empty if no slots
    return free_slots